In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model

# Get current working directory
cwd = os.getcwd()

# Load the saved model from disk
model = cwd + '/bank-note-recognition-model.h5'
reconstructed_model = load_model(model)

# Set image height and width
img_height = 150
img_width = 150

# Check that the model loads correctly
#reconstructed_model.summary()

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

# Create upload button
uploader = widgets.FileUpload(
    accept = 'image/jpeg, image/png',
    multiple = False
)

# Save uploaded file to tmp folder on disk
def on_file_upload(change):
    uploaded_file = next(iter(uploader.value))
    content = uploader.value[uploaded_file]['content']
    with open('./tmp/' + uploaded_file, 'wb') as f: f.write(content)

uploader.observe(on_file_upload, names='_counter')

# Instructions
print("Upload a single image of either a JMD $500 or JMD $1,000 bank note.\n")
print("After uploading, click on the Predict button.")

# Display button
display(uploader)

Upload a single image of either a JMD $500 or JMD $1,000 bank note.

After uploading, click on the Predict button.


FileUpload(value={}, accept='image/jpeg, image/png', description='Upload')

In [5]:
# Create predict button
predict_btn = widgets.Button(
    description = 'Predict',
    disabled = False,
    button_style = 'success',
    tooltip = 'Predict whether the uploaded image is a JMD $500 or JMD $1000 bank note',
    icon = ''
)

# Create output widget
output = widgets.Output()

# Function to get the filename of just uploaded file
def get_filename():
    if(uploader.data):
        uploaded_file = next(iter(uploader.value))
        filename = uploader.value[uploaded_file]['metadata']['name']
        
        return  cwd + '/tmp/' + filename
    else:
        return "No image has been uploaded"

# Function to determine whether the uploaded note is a JMD $500 or a JMD $1,000
def predict(uploaded_file):
    uploaded_file = get_filename()
    uploader.value.clear()
    
    with output:
        clear_output()
        
        img = keras.preprocessing.image.load_img(uploaded_file, target_size = (img_height, img_width))
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        predictions = reconstructed_model.predict(img_array)
        
        # Each prediction will be the output of the first class (jmd_500_bank note) being predicted
        # Therefore, the probability of the other class being predicted 1 - prob_first_class
        for i, predicted in enumerate(predictions):
            # JMD #500
            if predicted[0] > 0.50:
                print(
                    "This image is a JMD $500 bank note. Prob: {}"
                    .format(predicted[0])
                )
            
            # JMD #1000
            elif predicted[0] <= 0.50 and predicted[0] > 0.000000005:
                 print(
                     "This image is a JMD $1,000 bank note. Prob: {}"
                    .format(1 - predicted[0])
                 )
                    
            # None of the two
            else: print("Bank note not recognized. Prob: {}"
                    .format(predicted[0])
                )

# Listen for button click
predict_btn.on_click(predict)

# Display button
display(predict_btn, output)

Button(button_style='success', description='Predict', style=ButtonStyle(), tooltip='Predict whether the upload…

Output()